# BTO Cost Estimator Agent Data Cleaning

# Data Preparation
Parse BTO text copied from **HDB Annexes** on **BTO Flat Supply and Pricing Details**

In [13]:
import pandas as pd
import re
bto_df = pd.DataFrame()

In [14]:
def parse_bto_text(table_text):
    lines = [line.strip() for line in table_text.strip().split('\n') if line.strip()]
    data = []

    current_exercise = None
    current_project_type = None
    current_project = None
    current_wait_time = None

    for line in lines:
        # Detect exercise name (e.g., "Feb 2025")
        if re.match(r'^[A-Za-z]{3,9}\s+\d{4}$', line):
            current_exercise = line
            continue

        # Detect Project Type
        if line in ["Standard Projects", "Plus Project", "Prime Project"]:
            current_project_type = line
            continue

        # Detect Project + Waiting Time
        if "$" not in line and re.search(r'\d+(/?\d*)$', line):
            parts = line.rsplit(" ", 1)
            if len(parts) == 2:
                current_project = parts[0]
                current_wait_time = parts[1]
            else:
                current_project = line
            continue

        # Detect flat row (contains price)
        if "$" in line:
            # Extract price
            price_match = re.search(r'\$(\d{1,3}(?:,\d{3})*)\s*-\s*\$(\d{1,3}(?:,\d{3})*)', line)
            if not price_match:
                continue
            min_price = int(price_match.group(1).replace(",", ""))
            max_price = int(price_match.group(2).replace(",", ""))

            # Remove price part
            row_text = line[:price_match.start()].strip()
            tokens = row_text.split()

            # Detect flat type
            if len(tokens) > 1 and tokens[1].lower() == "flexi":
                flat_type = " ".join(tokens[:2])
                tokens = tokens[2:]
            else:
                flat_type = tokens[0]
                tokens = tokens[1:]

            # Start from the right to parse units & internal floor
            units = tokens[-1]

            # Check if internal floor is a range
            if "-" in tokens[-3:-1]:
                est_internal_area = " ".join(tokens[-4:-1])
                floor_tokens_end = -4
            else:
                est_internal_area = tokens[-2]
                floor_tokens_end = -2

            # Remaining tokens = floor area
            est_floor_area = " ".join(tokens[:floor_tokens_end])

            # Normalize ranges
            est_floor_area = re.sub(r"\s*-\s*", " - ", est_floor_area)
            est_internal_area = re.sub(r"\s*-\s*", " - ", est_internal_area)

            data.append({
                "Exercise": current_exercise,
                "Project Type": current_project_type,
                "Project": current_project,
                "Waiting Time": current_wait_time,
                "Flat Type": flat_type,
                "Estimated Floor Area": est_floor_area,
                "Estimated Internal Floor Area": est_internal_area,
                "Units": int(units),
                "Min Price": min_price,
                "Max Price": max_price
            })

    return pd.DataFrame(data)

In [15]:
table_text = """
Jul 2025
Standard Projects
Bangkit Breeze 35
2-room Flexi 41 38 24 $149,000 - $201,000
2-room Flexi 49 46 96 $180,000 - $236,000
3-room 69 66 96 $289,000 - $367,000
4-room 94 90 239 $362,000 - $458,000
5-room 114 110 188 $518,000 - $599,000
Sembawang Beacon 36
2-room Flexi 40 38 32 $148,000 - $173,000
2-room Flexi 48 46 136 $167,000 - $207,000
3-room 69 66 84 $267,000 - $323,000
4-room 94 90 286 $328,000 - $413,000
5-room 114 110 203 $487,000 - $586,000
3Gen 120 115 34 $497,000 - $585,000
Simei Symphony 42
2-room Flexi 40 38 40 $193,000 - $234,000
2-room Flexi 48 46 100 $223,000 - $280,000
4-room 93 90 140 $529,000 - $625,000
5-room 113 110 100 $658,000 - $763,000
Woodlands North Grove 55
2-room Flexi 40 38 64 $160,000 - $210,000
2-room Flexi 48 46 164 $184,000 - $250,000
3-room 69 66 88 $305,000 - $389,000
4-room 93 - 96 90 - 93 420 $388,000 - $544,000
5-room 113 - 116 110 - 113 412 $517,000 - $714,000
Prime Project
Alexandra Peaks 56
3-room 66 63 76 $403,000 - $511,000
4-room 89 86 422 $560,000 - $771,000
Alexandra Vista 47
2-room Flexi 40 38 62 $205,000 - $294,000
2-room Flexi 48 46 186 $266,000 - $355,000
3-room 67 64 93 $420,000 - $518,000
4-room 90 86 268 $547,000 - $775,000
Clementi Emerald 34 
2-room Flexi 41 38 66 $214,000 - $289,000
2-room Flexi 49 46 156 $261,000 - $351,000
3-room 67 64 111 $388,000 - $512,000
4-room 90 86 420 $562,000 - $780,000
Toa Payoh Ascent 41
2-room Flexi 38 36 39 $212,000 - $272,000
2-room Flexi 48 46 156 $258,000 - $354,000
3-room 66 63 78 $406,000 - $514,000
4-room 89 86 468 $583,000 - $777,000
"""

df = parse_bto_text(table_text)
df


,Exercise,Project Type,Project,Waiting Time,Flat Type,Estimated Floor Area,Estimated Internal Floor Area,Units,Min Price,Max Price
0,Jul 2025,Standard Projects,Bangkit Breeze,35,2-room Flexi,41,38,24,149000,201000
1,Jul 2025,Standard Projects,Bangkit Breeze,35,2-room Flexi,49,46,96,180000,236000
2,Jul 2025,Standard Projects,Bangkit Breeze,35,3-room,69,66,96,289000,367000
3,Jul 2025,Standard Projects,Bangkit Breeze,35,4-room,94,90,239,362000,458000
4,Jul 2025,Standard Projects,Bangkit Breeze,35,5-room,114,110,188,518000,599000
5,Jul 2025,Standard Projects,Sembawang Beacon,36,2-room Flexi,40,38,32,148000,173000
6,Jul 2025,Standard Projects,Sembawang Beacon,36,2-room Flexi,48,46,136,167000,207000
7,Jul 2025,Standard Projects,Sembawang Beacon,36,3-room,69,66,84,267000,323000
8,Jul 2025,Standard Projects,Sembawang Beacon,36,4-room,94,90,286,328000,413000
9,Jul 2025,Standard Projects,Sembawang Beacon,36,5-room,114,110,203,487000,586000


In [ ]:
bto_df = pd.concat([bto_df, df], ignore_index=True)
bto_df

,Exercise,Project Type,Project,Waiting Time,Flat Type,Estimated Floor Area,Estimated Internal Floor Area,Units,Min Price,Max Price
0,Jul 2025,Standard Projects,Bangkit Breeze,35,2-room Flexi,41,38,24,149000,201000
1,Jul 2025,Standard Projects,Bangkit Breeze,35,2-room Flexi,49,46,96,180000,236000
2,Jul 2025,Standard Projects,Bangkit Breeze,35,3-room,69,66,96,289000,367000
3,Jul 2025,Standard Projects,Bangkit Breeze,35,4-room,94,90,239,362000,458000
4,Jul 2025,Standard Projects,Bangkit Breeze,35,5-room,114,110,188,518000,599000
...,...,...,...,...,...,...,...,...,...,...
230,Oct 2023,Prime Project,Verandah @ Kallang,42,2-room Flexi,49,46,112,240000,303000
231,Oct 2023,Prime Project,Verandah @ Kallang,42,3-room,69,66,110,368000,475000
232,Oct 2023,Prime Project,Verandah @ Kallang,42,4-room,93,90,893,535000,675000
233,Oct 2023,Prime Project,Tanglin Halt Cascadia,54,3-room,66,63,155,364000,509000


In [ ]:
# Export to CSV
# bto_df = pd.read_csv("bto_pricing_detail.csv")
# bto_df.to_csv("bto_pricing_detail.csv", index=False)

# Data Cleaning

In [16]:
import pandas as pd
import numpy as np
import re

def parse_range(value):
    """parse a numeric range like '93 - 96' and return the average as float"""
    if pd.isna(value):
        return np.nan
    if isinstance(value, (int, float)):
        return float(value)
    # match "min - max" or "min–max" (handle dash variants)
    match = re.match(r'(\d+(?:\.\d+)?)\s*[-–]\s*(\d+(?:\.\d+)?)', str(value))
    if match:
        low, high = match.groups()
        return (float(low) + float(high)) / 2
    # if single number
    try:
        return float(value)
    except ValueError:
        return np.nan

def clean_bto_data(csv_path: str) -> pd.DataFrame:
    """load and clean bto pricing"""
    df = pd.read_csv(csv_path)

    # strip column names and lower-case
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

    # normalize key text columns
    for col in ["project", "flat_type", "project_type"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.lower()

    # parse exercise as datetime (assume format like "Feb 2025")
    if "exercise" in df.columns:
        df["exercise_date"] = pd.to_datetime(df["exercise"], format="%b %Y", errors="coerce")

    # parse numeric ranges in floor area columns
    for col in ["estimated_floor_area", "estimated_internal_floor_area"]:
        if col in df.columns:
            df[col] = df[col].apply(parse_range)

    # ensure numeric columns
    for col in ["units", "min_price", "max_price"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # compute average price
    if "min_price" in df.columns and "max_price" in df.columns:
        df["avg_price"] = df[["min_price", "max_price"]].mean(axis=1)

    # optional: compute average floor area (external + internal)
    if "estimated_floor_area" in df.columns and "estimated_internal_floor_area" in df.columns:
        df["avg_floor_area"] = df[["estimated_floor_area", "estimated_internal_floor_area"]].mean(axis=1)

    # drop rows with no useful price data
    df = df.dropna(subset=["avg_price"])

    return df


In [22]:
df_clean = clean_bto_data("bto_pricing_detail.csv")
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   exercise                       235 non-null    object        
 1   project_type                   235 non-null    object        
 2   project                        235 non-null    object        
 3   waiting_time                   235 non-null    object        
 4   flat_type                      235 non-null    object        
 5   estimated_floor_area           235 non-null    float64       
 6   estimated_internal_floor_area  235 non-null    float64       
 7   units                          235 non-null    int64         
 8   min_price                      235 non-null    int64         
 9   max_price                      235 non-null    int64         
 10  exercise_date                  235 non-null    datetime64[ns]
 11  avg_price          

In [23]:
# save to new csv
df_clean.to_csv("bto_pricing_detail_cleaned.csv", index=False)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   exercise                       235 non-null    object        
 1   project_type                   235 non-null    object        
 2   project                        235 non-null    object        
 3   waiting_time                   235 non-null    object        
 4   flat_type                      235 non-null    object        
 5   estimated_floor_area           235 non-null    float64       
 6   estimated_internal_floor_area  235 non-null    float64       
 7   units                          235 non-null    int64         
 8   min_price                      235 non-null    int64         
 9   max_price                      235 non-null    int64         
 10  exercise_date                  235 non-null    datetime64[ns]
 11  avg_price          